# SemEval-2018 Task 1: Affect in Tweets (AIT-2018)

## An emotion intensity ordinal classification task

*Joana Ferreira |
joanaferreira0011@gmail.com |
Faculty of Engineering, University of Porto, R. Dr. Roberto Frias, 4200-465 Porto, Portugal *

### Abstract:
This notebook presents a solution for the SemEval-2018 Task 1: Affect in Tweets (AIT-2018). Given a tweet and an emotion (sadness, anger, fear, joy), the model should output an intensity classification (0, 1, 2, 3). To do this, two different approaches were implemented: the first one using a knowledge based preprocessing the second one using word embeddings (BERT). Finally, several models were used and compared to do the classification.


### 1. Introduction
Given a tweet and an emotion (sadness, anger, fear or joy), the proposed task is to classify it in one of four classes given its intensity: (0:no,
1: low, 2: moderate, 3: high). The task was solved only for the English language. 

This task was proposed in (Mohammad et al.,2018).

### 2. Description of the dataset
There are four training and test sets of labeled data – one for each emotion. The data creation is described in (Mohammad and Kiritchenko, 2018).

The training and test sets were merged together and then split (20/80) in order to increase the size of the dataset for training.

In [ ]:
import pandas as pd

li = []

#Change this the correct path of the desired dataset for a specific emotion
all_files= {'../datasets/2018-EI-oc-En-anger-dev.txt', '../datasets/EI-oc-En-anger-train.txt'}

for filename in all_files:
    df = pd.read_csv(filename, sep="\t", header=None, skiprows=1)
    li.append(df)

dataset = pd.concat(li, axis=0, ignore_index=True)

dataset.columns = ['date', 'text', 'emotion', 'level']
dataset= dataset.sample(frac=1)
print (dataset)


### 3. Approach 

Two alternatives were implemented for preprocessing: a knowledge based approach (section 3.1.1) and one using word embeddings (BERT) (section 3.1.2). After the preprocessing, both data were run for the same models: Naive Bayes, SVM, Logistic regression, Multi-layer Perceptron classifier, Perceptron, Decision Tree and Random Forest.

All of the models and preprocessing were run for each emotion seperately, as the emotion is an input of the problem. 

#### 3.1.1. Preprocessing using a knowledge based approach
In this approach, the following processes were implemented:
* **Tokenization**: tokens were created using Python String split() method. Other tokenization methods were tested such as NLTK word_tokenize(), but no significant improvements were observed.
* **Lower case**: tokens were converted to lower case
* **Lemmatization**: using NLTK WordNetLemmatizer
* **User**: All user mentions were substituted with '@user'
* **Emojis**: All emojis were converted to keywords, using the 'emoji' library
* **TF-IDF**: SkLearn's TfidfVectorizer was used to perform TF-IDF
* **Bag of Words (BoW)**: Bag of words was used but eventually substitute with TF-IDF, because of the better overall accuracy


*Skip to section 3.1.2. and do not run the following 3 cells if you want to see the results from using word embeddings (BERT).*


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
import emoji
from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 

corpus = []
ps = PorterStemmer()

for index, row in dataset.iterrows():
    tweet = row['text']

    #handle users
    tweet = re.sub('@.*', '@user', tweet) 

    
    
    tweet = tweet.tolower().split()
    #tweet = nltk.word_tokenize(tweet) - can be used but a few changes need to be done in the following lines because the data structures will change

    # lemmatization and stop word removal
    tweet = ' '.join([lemmatizer.lemmatize(w) for w in tweet if not w in set(stopwords.words('english'))])
    

    #tweet = nlp(tweet) # run annotation over a sentence
    
    
    #emojis
    tweet = emoji.demojize(tweet)
    
    corpus.append(tweet)

print(corpus)


In [ ]:
#TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer=TfidfVectorizer(use_idf=True)
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(corpus)

print(tfidf_vectorizer_vectors)

In [ ]:
import numpy as np


X = tfidf_vectorizer_vectors.toarray()
y = []
for index, row in dataset.iterrows():
    y.append(int(row['level'][0]))

    #Uncomment the following lines and comment the one before to reduce the number of classes to a binary classification
    '''
    if(int(row['level'][0])>=2):
        y.append(1) 
    else:
        y.append(0)
    '''
    
    
y = np.array(y)

print(X, y)

#### 3.1.2. Preprocessing using BERT
In this approach, word embedding was used to preprocess the data. For this, BERT was chosen and the library *bert_embedding* was used because of its simplicity. The model was pre-trained in the following dataset: *book_corpus_wiki_en_cased*.

*Skip to section 3.2 and do not run the following 3 cells if you want to see the results from using the knowledge based approach described in 3.1.1.*


In [ ]:
from bert_embedding import BertEmbedding
import re
import nltk
import emoji
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import mxnet as mx


corpus = []

for index, row in dataset.iterrows():
    tweet = row['text']
    corpus.append(tweet)

bert = BertEmbedding(model='bert_24_1024_16', dataset_name='book_corpus_wiki_en_cased')
results = bert(corpus)
print(results)

In [ ]:
import numpy as np
averaged = []
for sent in results:
    averaged.append(np.mean(sent[1], axis = 0, dtype=np.float64))

corpus=averaged
X=np.array(corpus)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

y = []
for index, row in dataset.iterrows():
    y.append(int(row['level'][0]))
    
    #Uncomment the following lines and comment the one before to reduce the number of classes to a binary classification
    '''
    if(int(row['level'][0])>=2):
        y.append(1) 
    else:
        y.append(0)
    '''
    
    
y = np.array(y)
print(X, y)

### 3.2. Classification
After the preprocessing, the data was run for the following models (All from SkLearn). 
* Naive Bayes 
* Support Vector Machine(SVM)
* Logistic regression
* Multi-layer Perceptron classifier
* Perceptron
* Decision Tree
* Random Forest

To evaluate and compare them, Accuracy, Precision, Recall and F1 were measured.


In [ ]:
# Split dataset into training and test sets

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
# Naive Bayes

from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

classifier = GaussianNB()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='weighted'))
print('Recall: ', recall_score(y_test, y_pred, average='weighted'))
print('F1: ', f1_score(y_test, y_pred, average='weighted'))

In [ ]:
# SVM

from sklearn.svm import SVC

classifier = SVC()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='weighted'))
print('Recall: ', recall_score(y_test, y_pred, average='weighted'))
print('F1: ', f1_score(y_test, y_pred, average='weighted'))

In [ ]:
# Logistic Regression

from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='weighted'))
print('Recall: ', recall_score(y_test, y_pred, average='weighted'))
print('F1: ', f1_score(y_test, y_pred, average='weighted'))

In [ ]:
# SGDC classifier

from sklearn.linear_model import SGDClassifier

classifier = SGDClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='weighted'))
print('Recall: ', recall_score(y_test, y_pred, average='weighted'))
print('F1: ', f1_score(y_test, y_pred, average='weighted'))

In [ ]:
from sklearn.neural_network import MLPClassifier

classifier = MLPClassifier(random_state=1, max_iter=300)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='weighted'))
print('Recall: ', recall_score(y_test, y_pred, average='weighted'))
print('F1: ', f1_score(y_test, y_pred, average='weighted'))

In [ ]:
# Perceptron
from sklearn.linear_model import Perceptron

classifier = Perceptron() 
classifier.fit(X_train, y_train) 
y_pred = classifier.predict(X_test)

print(confusion_matrix(y_test, y_pred)) 
print('Accuracy: ', accuracy_score(y_test, y_pred)) 
print('Precision: ', precision_score(y_test, y_pred, average='weighted')) 
print('Recall: ', recall_score(y_test, y_pred, average='weighted')) 
print('F1: ', f1_score(y_test, y_pred, average='weighted'))

In [ ]:
# Decision Tree

from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='weighted'))
print('Recall: ', recall_score(y_test, y_pred, average='weighted'))
print('F1: ', f1_score(y_test, y_pred, average='weighted'))

In [ ]:
# Random Forest

from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred, average='weighted'))
print('Recall: ', recall_score(y_test, y_pred, average='weighted'))
print('F1: ', f1_score(y_test, y_pred, average='weighted'))

### 4. Experimental evaluation

#### 4.1 Accuracy

The following values for the accuracy were obtain as the median of 3 runs for each model, using Sklearn's *accuracy_score*.

#### 4.1.1 Joy 

| Models | Naive Bayes | SVM | Logistic Regression | MLPClassifier | Perceptron | Decision tree | Random forest |
| --- | --- | --- | --- | --- | --- | --- | --- |
| Knowledge | 0.346 | **0.419** | 0.408 | 0.369 | 0.397 | 0.390 | 0.408 |
| BERT | 0.471 | 0.440 | 0.424 | 0.458 | 0.400 | 0.372 | **0.476** |

#### 4.1.2 Sadness

| Models | Naive Bayes | SVM | Logistic Regression | MLPClassifier | Perceptron | Decision tree | Random forest |
| --- | --- | --- | --- | --- | --- | --- | --- |
| Knowledge | 0.326 | 0.436 | 0.471 | 0.443 | **0.474** | 0.463 | **0.474** |
| BERT | 0.409 | **0.489** | 0.437 | 0.472 | 0.409 | 0.319 | 0.432 |

#### 4.1.3 Anger

| Models | Naive Bayes | SVM | Logistic Regression | MLPClassifier | Perceptron | Decision tree | Random forest |
| --- | --- | --- | --- | --- | --- | --- | --- |
| Knowledge | 0.284 | 0.430 | **0.447** | 0.411 | 0.406 | 0.409 | **0.447** |
| BERT | 0.411 | **0.480** | 0.450 | 0.483 | 0.428 | 0.378 | 0.443 |

#### 4.1.4 Fear

| Models | Naive Bayes | SVM | Logistic Regression | MLPClassifier | Perceptron | Decision tree | Random forest |
| --- | --- | --- | --- | --- | --- | --- | --- |
| Knowledge | 0.319 | 0.677 | 0.674 | 0.633 | 0.686 | 0.640 | **0.691** |
| BERT | 0.539 | **0.681** | 0.621 | 0.671 | 0.639 | 0.533 | 0.665 |

#### 4.2 Confusion Matrices

For each run 4 metrics were calculated using Sklearn: accuracy, precision, recall and F1.
 
Most confusion matrices obtained showed that when the prediction was wrong, most algorithms actually predicted one of the closests classes to the expected one. For instance, when expected class 0, if the prediction was wrong the algorithm would more likely have predicted 1, than 3 or 4.

An example of one matrix can be found here:

| Class | 0 | 1 | 2 | 3 |
| --- | --- | --- | --- | --- |
| 0 | 56 | 36 | 16 | 19 | 
| 1 | 30 | 15 | 25 | 7 | 
| 2 | 21 | 31 |36 | 29 | 
| 3 | 12 | 9 | 11 | 65 | 

Accuracy:  0.41148325358851673 | 
Precision:  0.4135462119735973 | 
Recall:  0.41148325358851673 | 
F1:  0.40855124453720665 |

*Confusion matrix and metrics for one run - emotion Anger: BERT with Naive Bayes*


#### 4.3 Fewer classes

Given the observations taken from the confusion matrices, the algortithms were run again but in for a binary classification. 
* Classes 0 and 1 converged to Class 0
* Classes 2 and 3 converged to Class 1

Given this simplification, most accuracies increased by around 20%. This corroborates the idea that the models have some difficulty distinguishing classes that are closer to each other.





### 5. Conclusions

After analysing the results, we conclude that the best results were mostly obtained by using BERT. Reggarding the models, SVM and Random Forest tend to outperform the rest for almost all the tasks. 

In the future, other BERT algorithms and embeddings should be tested and a convulutional neural network (CNN) should also be implemented. Fine tuning the models even further with either types of preprocessing should also contribute to a better overall accuracy.

### References

Priban, Pavel & Hercig, Tomáš & Lenc, Ladislav. (2018). UWB at SemEval-2018 Task 1: Emotion Intensity Detection in Tweets. 133-140. 10.18653/v1/S18-1018.